###  MicroGrad demo

In [70]:
from micrograd.engine import Value
from micrograd.nn import AutoEncoder, VariationalAutoEncoder
from micrograd.loss_funcs import mean_squared_error, vae_loss
from micrograd.optimizer import SGD
import numpy as np
import random

In [71]:
np.random.seed(1337)
random.seed(1337)
np.set_printoptions(suppress=True)

In [72]:
x = [
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1],
]
target = x

In [73]:
auto = AutoEncoder(
    in_embeds=5,# input dimension   , # len = number of layers, i = size of layer
    hidden_layers=[3],
    latent_dim=2, # compressed layer dimensions
    act_func=Value.sigmoid, # activation function for final decoder layer
)

optimizer = optimizer = SGD(auto.parameters(), learning_rate=0.5)

In [74]:
for i in range(2000):
    optimizer.zero_grad()
    
    reconstruction = [auto(i) for i in x]
    
    loss = sum([mean_squared_error(recon, targ) for recon, targ in zip(reconstruction, target)])

    # Backward pass
    loss.backward()
    
    optimizer.step()
    


    if i % 10 == 0:
        print(f"Iteration: {i}, AutoEncoder Loss: {loss.data:.12f}")

Iteration: 0, AutoEncoder Loss: 1.371534565316
Iteration: 10, AutoEncoder Loss: 0.828867493523
Iteration: 20, AutoEncoder Loss: 0.800121316840
Iteration: 30, AutoEncoder Loss: 0.796200928183
Iteration: 40, AutoEncoder Loss: 0.793153087901
Iteration: 50, AutoEncoder Loss: 0.789298019836
Iteration: 60, AutoEncoder Loss: 0.783577204890
Iteration: 70, AutoEncoder Loss: 0.774073506406
Iteration: 80, AutoEncoder Loss: 0.757584390492
Iteration: 90, AutoEncoder Loss: 0.720449213257
Iteration: 100, AutoEncoder Loss: 0.650519424161
Iteration: 110, AutoEncoder Loss: 0.601960645456
Iteration: 120, AutoEncoder Loss: 0.574013927675
Iteration: 130, AutoEncoder Loss: 0.556683287104
Iteration: 140, AutoEncoder Loss: 0.545111582130
Iteration: 150, AutoEncoder Loss: 0.537140917181
Iteration: 160, AutoEncoder Loss: 0.530531654328
Iteration: 170, AutoEncoder Loss: 0.525172208415
Iteration: 180, AutoEncoder Loss: 0.520056642697
Iteration: 190, AutoEncoder Loss: 0.518232457409
Iteration: 200, AutoEncoder Los

In [75]:
for i in range(len(x[0])):
    for j in range(len(x)):
        print(f"expected {x[j][i]}, result, {auto(x[j])[i].data}")
    print("--------------------------------")

expected 1, result, 0.9828378220266868
expected 0, result, 0.012172861565287187
expected 0, result, 3.901187256193221e-47
expected 0, result, 0.005003588986116644
expected 0, result, 6.857949047430323e-09
--------------------------------
expected 0, result, 0.005675473893101285
expected 1, result, 0.9898491343551946
expected 0, result, 6.2553952232554994e-77
expected 0, result, 1.9639129351845353e-47
expected 0, result, 3.83708905981204e-05
--------------------------------
expected 0, result, 0.005871274840454873
expected 0, result, 0.001939028395184959
expected 1, result, 0.9851282356482717
expected 0, result, 0.014210616354544917
expected 0, result, 0.037494859788822464
--------------------------------
expected 0, result, 0.039206799781306576
expected 0, result, 0.011868570847641412
expected 0, result, 0.009582516696300777
expected 1, result, 0.9842390069729133
expected 0, result, 0.0023686139937995885
--------------------------------
expected 0, result, 0.0005334597907096435
expecte

In [79]:
vae = VariationalAutoEncoder(
    in_embeds=10,  
    hidden_layers=[7],
    latent_dim=4,     
    act_func=Value.sigmoid
)

vae_x = [Value(random.uniform(0, 1)) for _ in range(10)]

vae_target = vae_x

vae_optimizer = SGD(vae.parameters(), learning_rate=0.1)

In [77]:
for i in range(1000):
    vae_optimizer.zero_grad()

    reconstruction, mu, log_var = vae(vae_x)

    if not isinstance(reconstruction, list):
        reconstruction = [reconstruction]


    loss = vae_loss(reconstruction, vae_target, mu, log_var)

    # Backward pass
    loss.backward() 

    
    vae_optimizer.step()

    
    if i % 10 == 0:
        print(f"Iteration {i}, VAE Loss: {loss.data:.12f}")





Iteration 0, VAE Loss: 0.170626104863
Iteration 10, VAE Loss: 0.131525882735
Iteration 20, VAE Loss: 0.108139044673
Iteration 30, VAE Loss: 0.096432586365
Iteration 40, VAE Loss: 0.097865735823
Iteration 50, VAE Loss: 0.074479838637
Iteration 60, VAE Loss: 0.061534849368
Iteration 70, VAE Loss: 0.094860216655
Iteration 80, VAE Loss: 0.061495898405
Iteration 90, VAE Loss: 0.085765781855
Iteration 100, VAE Loss: 0.059986325212
Iteration 110, VAE Loss: 0.077054973399
Iteration 120, VAE Loss: 0.078699398175
Iteration 130, VAE Loss: 0.069944847449
Iteration 140, VAE Loss: 0.048835501760
Iteration 150, VAE Loss: 0.060075495213
Iteration 160, VAE Loss: 0.060551678856
Iteration 170, VAE Loss: 0.046948542038
Iteration 180, VAE Loss: 0.045171046028
Iteration 190, VAE Loss: 0.049456701024
Iteration 200, VAE Loss: 0.041387394008
Iteration 210, VAE Loss: 0.056881396611
Iteration 220, VAE Loss: 0.049644293772
Iteration 230, VAE Loss: 0.030230020479
Iteration 240, VAE Loss: 0.028289717526
Iteration 2

In [78]:
noise = np.random.normal(size=(10))

generated_samples = vae.decode(noise)
for i in range(len(noise)):
    print(f"Actual Value: {vae_x[i].data}, generated Value: {generated_samples[i].data}")


Actual Value: 0.484368020718412, generated Value: 0.556659056499744
Actual Value: 0.151270886744849, generated Value: 0.09377571083477061
Actual Value: 0.10854135277607113, generated Value: 0.13579411652640058
Actual Value: 0.2047944506633067, generated Value: 0.1931632424309115
Actual Value: 0.8996515210360224, generated Value: 0.8766529052359195
Actual Value: 0.7044760413408526, generated Value: 0.7176015852633344
Actual Value: 0.1631162699452855, generated Value: 0.14863566238654993
Actual Value: 0.043721800004570266, generated Value: 0.13885014252413172
Actual Value: 0.0022668172091712124, generated Value: 0.06705478010374977
Actual Value: 0.2372607572927734, generated Value: 0.14875811361140243
